In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import openpyxl
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense,Dropout, Conv1D,GlobalMaxPooling1D,concatenate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## 데이터 임포트

In [2]:
df= pd.read_excel('./감성대화말뭉치(원시데이터)_Training.xlsx')
df2 = pd.read_excel("./감성대화말뭉치(원시데이터)_Validation.xlsx")
df3 = pd.read_excel("./감성대화말뭉치(최종데이터)_Training.xlsx")
df4 = pd.read_excel("./감성대화말뭉치(최종데이터)_Validation.xlsx")
df5 = pd.concat([df,df2,df3,df4],axis=0)
df5 = df5.drop(columns=['사람문장4','시스템응답4','신체질환'])
df5 = df5.drop_duplicates().reset_index()
df5 = df5.drop(columns=['index'])
df5 = df5.drop(4)
df5 = df5.reset_index().drop(columns='index')
df5 = df5.fillna("").drop_duplicates()
np.unique(df5['감정_대분류'],return_counts=True)
emo_dic = {
    "기쁨" : 0,    "기쁨 " : 0,    "분노" : 1,    "불안" : 2,    "불안 " : 2,    "슬픔" : 3,
}
env_dic={
    "가족관계" : 0 ,    
    "건강" : 1,    
    '건강, 죽음': 1 ,    
    '대인관계': 2 , 
    "대인관계(부부, 자녀)" : 2 ,
    "연애, 결혼, 출산" : 3 ,    
    "재정" : 4 ,    "재정, 은퇴, 노후준비" : 4 , 
    "직장, 업무 스트레스" : 5 ,  
    "진로, 취업, 직장" : 5 ,    
    "학교폭력/따돌림" : 6 ,    
    "학업 및 진로" : 5 
}


df5['상황키워드'] = df5['상황키워드'].map(env_dic)
df5['감정_대분류'] = df5['감정_대분류'].map(emo_dic)

sen1 = df5['사람문장1']
sen2 = df5['사람문장2']
sen3 = df5['사람문장3']
full_sen = []
for i in range(0,len(df5)):
    txt = str(sen1[i])+str(sen2[i])+str(sen3[i])
    full_sen.append(txt)
df5['전체문장'] = full_sen
df5.to_excel("8만여개의 감성데이터.xlsx")

In [49]:
result = pd.read_excel("./0620.xlsx")
result['상황키워드'] = result['상황키워드'].map(env_dic)
result['감정_대분류'] = result['감정_대분류'].map(emo_dic)
result

,연령,상황키워드,감정_대분류,사람문장1,시스템응답1
0,노년,1,2.0,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약이 많아서 힘드시겠어요. 줄이기 위해서는 어떻게 해야 하나요?
1,노년,4,2.0,너무 안 좋은데 하기 싫고 하자니 비싸네.,그렇군요. 그래도 필요하다면 해야겠죠?
2,노년,2,2.0,대인관계에 혼란을 느껴.,무슨 일이 있으셧나요?. 어떤 일 때문에 그러세요?
3,노년,1,2.0,몸이 안좋으데 이제 어떻게 살아가야 하는지 . 많이 혼란스럽네.,걱정이 많으시군요. 다른 방법은 없을까요?
4,노년,1,2.0,수술해 본 적이 없는데 신장 이식 수술을 받아야 한다고 해서 무서워.,많이 두려우실것 같아요. 그래도 좋은 결과 있기를 바랍니다
...,...,...,...,...,...
24247,중년,1,0.0,우리 가족 건강은 내가 책임지고 있어. 왜냐면 내가 건강하거든.,기분이 좋으시겠어요.
24248,중년,5,0.0,오늘 회사에서 새 프로젝트를 성사시켰어.,좋은 결과를 얻어 기쁘시겠어요.
24249,중년,4,0.0,은퇴 후에 취미생활하면서 나한테 투자하고 있어. 너무 좋아.,기쁘시겠어요.
24250,중년,5,0.0,나는 우리 부서가 올해 최대 실적을 거둘 거로 생각하고 있어.,그렇게 생각하게 된 계기가 있나요?


In [4]:
np.unique(df5.상황키워드,return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6], dtype=int64),
 array([ 4742, 12546, 20874,  7617, 12136, 18977,  5046], dtype=int64))

In [5]:
features = df5['전체문장'].dropna().to_list()
labels = df5['감정_대분류'].to_list()
len(features), len(labels)

(81938, 81938)

### 감성대화 말뭉치 데이터 
- 4개 병합 
- 감정mapping
- 사람문장1,2,3  합치기

### 1. 훈련데이터, 정답데이터 추출( 사람문장1만 사용 )
- 훈련데이터 : 전체문장 
- 정답데이터 : 감정_대분류

In [6]:
df5 = pd.read_excel("./8만여개의 감성데이터.xlsx")
features = df5['전체문장'].dropna().to_list()
labels = df5['감정_대분류'].to_list()
len(features), len(labels)

(81938, 81938)

### 단어사전 구축 
- text_to_word_sequence : 132,667개
- Okt : 55,692개
- komoran : 18,274개


<br><br>


### 1-3 전처리
- word_index : 총 단어 수 
- MAX_SEQ_LEN : 문장 길이 고정 
- padded_seqs : 문장 길이 정규화

In [7]:
from konlpy.tag import Okt
okt = Okt()
corpus = [okt.morphs(x) for x in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)

sequence = tokenizer.texts_to_sequences(corpus)

len_txt = [len(x) for x in sequence]

word_index = tokenizer.word_index
MAX_SEQ_LEN = 100  ##패딩 50
padded_seqs = preprocessing.sequence.pad_sequences(sequence, maxlen=MAX_SEQ_LEN)

# 1-4 CNN을 이용한 감정 분류 모델

In [10]:
ds = tf.data.Dataset.from_tensor_slices((padded_seqs,labels))
ds = ds.shuffle(len(features))
train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size+val_size).take(test_size).batch(20)

dropout_prob = 0.5
EMB_SIZE = 128
EPOCH=200
VOCAB_SIZE = len(word_index)+1

input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length = MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters=128,   kernel_size = 3, padding="valid",  activation = tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPooling1D()(conv1)

conv2 = Conv1D(filters=128, kernel_size = 4, padding="valid", activation = tf.nn.relu)(dropout_emb)
poo2 = GlobalMaxPooling1D()(conv2)

conv3 = Conv1D(filters=128, kernel_size = 5, padding="valid", activation = tf.nn.relu)(dropout_emb)
poo3 = GlobalMaxPooling1D()(conv3)

# 3,4,5-gram 이후 합치기 
concat = concatenate([pool1,poo2,poo3])

hidden = Dense(128,activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(4,name="logits")(dropout_hidden)
predictions = Dense(4, activation=tf.nn.softmax)(logits)

model = Model(inputs=input_layer, outputs = predictions)
model.compile(optimizer="adam",
             loss="sparse_categorical_crossentropy",
             metrics="accuracy")
model.summary()

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-emo-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,
                                                 restore_best_weights=True)

model.fit(train_ds, validation_data=val_ds, epochs=EPOCH,callbacks=[checkpoint_cb,early_stopping_cb])

# 모델 평가
loss, accuracy = model.evaluate(test_ds)
print("Accuracy : ",accuracy*100)
print("loss : ",loss)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 128)     7128704     ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 100, 128)     0           ['embedding[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 98, 128)      49280       ['dropout[0][0]']                
                                                                                              

# 1-5 CNN을 이용한 상황 분류 모델

In [8]:
df5 = pd.read_excel("./8만여개의 감성데이터.xlsx")
features = df5['전체문장'].dropna().to_list()
labels2 = df5['상황키워드'].to_list()
len(features), len(labels2)

(81938, 81938)

In [9]:
ds = tf.data.Dataset.from_tensor_slices((padded_seqs,labels2))
ds = ds.shuffle(len(features))
train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size+val_size).take(test_size).batch(20)

dropout_prob = 0.5
EMB_SIZE = 128
EPOCH=15
VOCAB_SIZE = len(word_index)+1

input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length = MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters=128,   kernel_size = 3, padding="valid",  activation = tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPooling1D()(conv1)

conv2 = Conv1D(filters=128, kernel_size = 4, padding="valid", activation = tf.nn.relu)(dropout_emb)
poo2 = GlobalMaxPooling1D()(conv2)

conv3 = Conv1D(filters=128, kernel_size = 5, padding="valid", activation = tf.nn.relu)(dropout_emb)
poo3 = GlobalMaxPooling1D()(conv3)

# 3,4,5-gram 이후 합치기 
concat = concatenate([pool1,poo2,poo3])

hidden = Dense(128,activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(7,name="logits")(dropout_hidden)
predictions = Dense(7, activation=tf.nn.softmax)(logits)

env_model = Model(inputs=input_layer, outputs = predictions)
env_model.compile(optimizer="adam",
             loss="sparse_categorical_crossentropy",
             metrics="accuracy")

checkpoint_cb2 = keras.callbacks.ModelCheckpoint('best-env-model.h5')
early_stopping_cb2 = keras.callbacks.EarlyStopping(patience=3,
                                                 restore_best_weights=True)

env_model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, callbacks=[checkpoint_cb2,early_stopping_cb2])

# 모델 평가
loss, accuracy = env_model.evaluate(test_ds)
print("Accuracy : ",accuracy*100)
print("loss : ",loss)

Epoch 1/15
2868/2868 [==============================] - 34s 10ms/step - loss: 0.8969 - accuracy: 0.6974 - val_loss: 0.5206 - val_accuracy: 0.8250
Epoch 2/15
 716/2868 [======>.......................] - ETA: 18s - loss: 0.6141 - accuracy: 0.8017

KeyboardInterrupt: 

### 1-6. 랜덤 사용자 문항에 대한 감정 예측

In [10]:
emo_list = ["기쁨","분노","불안","슬픔"]   # 감정클래스 리스트 

import random as rd    
num = rd.randrange(0,100)  # 데이터 프레임 행 중에 난수를 하나 생성

s1 = [result.iloc[20004,:]['사람문장1']]
cor_s1 = [okt.morphs(x) for x in s1]
seq_s1 = tokenizer.texts_to_sequences(cor_s1)
pad_s1 = preprocessing.sequence.pad_sequences(seq_s1,maxlen=MAX_SEQ_LEN)

print(s1)
print(cor_s1)
print(seq_s1)
print(pad_s1)

emo_model_predict_proba = model.predict(pad_s1)[0]
emo_model_predict_class_index = emo_list[np.argmax(model.predict(pad_s1)[0])]
emo_model_predict_index = np.argmax(model.predict(pad_s1)[0])
print("감정 예측 점수 : ",emo_model_predict_proba)
print("감정 예측 클래스 : ",emo_model_predict_class_index)
print("상황 예측 인덱스 : ",emo_model_predict_index)

['과중한 업무가 끝나서 이제 쉬려나 했는데 다시 업무가 부여가 돼서 지치네.']
[['과중', '한', '업무', '가', '끝나서', '이제', '쉬려나', '했는데', '다시', '업무', '가', '부여', '가', '돼서', '지치네', '.']]
[[5618, 27, 160, 4, 2757, 60, 29475, 55, 173, 160, 4, 11465, 4, 313, 4581, 1]]
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
   5618    27   160     4  2757    60 29475    55   173   160     4 11465
      4   313  4581     1]]


NameError: name 'model' is not defined

### 1-7. 랜덤 사용자 문항에 대한 상황 분류 예측

In [11]:
env_list = ["가족관계", "건강","대인관계","연애, 결혼, 출산","재정","직장","학교폭력/따돌림"]

s1 = [result.iloc[9656,:]['사람문장1']]
cor_s1 = [okt.morphs(x) for x in s1]
seq_s1 = tokenizer.texts_to_sequences(cor_s1)
pad_s1 = preprocessing.sequence.pad_sequences(seq_s1,maxlen=MAX_SEQ_LEN)


env_model_predict_proba = env_model.predict(pad_s1)[0]
env_model_predict_class = env_list[np.argmax(env_model.predict(pad_s1)[0])]
env_model_predict_index = np.argmax(env_model.predict(pad_s1)[0])
print("상황 예측 점수 : ",env_model_predict_proba)
print("상황 예측 클래스 : ",env_model_predict_class)
print("상황 예측 인덱스 : ",env_model_predict_index)

1/1 [==============================] - 0s 14ms/step
상황 예측 점수 :  [8.41859961e-04 1.81968901e-02 6.04156494e-01 3.32210981e-03
 5.99822868e-03 3.67372006e-01 1.12350746e-04]
상황 예측 클래스 :  대인관계
상황 예측 인덱스 :  2


### 유사도 함수

In [12]:
# 해당되는 감정과 카테고리 행을 추출하는 함수 
def emo_env_request(df, emotion, enviroment):
    condition1 = (df['감정_대분류']==emotion)
    condition2 = (df['상황키워드']==enviroment)
    Q_Answers = df[condition1 & condition2]
    return Q_Answers['사람문장1'].to_list() , Q_Answers['시스템응답1'].to_list()

# 감정문장 리스트 중 가장 유사한 문장을 찾아 인덱스를 반환하는 함수
def cosine_Answer(txt,df_list):
    cos_sim_list = [] 
    ####### 텍스트 유사도 측정 ######
    # 문장 벡터화 하기(사전 만들기)
    for i in df_list:
        tfidf_vectorizer = TfidfVectorizer()
        sentences = (txt,i)
        tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
        ### 코사인 유사도 ###
        cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
        cos_sim_list.append(cos_similar[0][0])
    # 가장 유사한 문장이 있는 인덱스와 가장 유사한 문장을 반환 
    return np.argmax(cos_sim_list), df_list[np.argmax(cos_sim_list)]


# 코사인 함수의 인덱스를 이용해 가장 적절한 응답을 찾는 함수 
def adaquate_answer(df, idx):
    return df[idx]

request, response = emo_env_request(result,emo_model_predict_index,env_model_predict_index)
idx, answer = cosine_Answer(s1[0],request)
moa_answer = adaquate_answer(response,idx)
print(s1[0])
print(answer)
print(moa_answer)

NameError: name 'emo_model_predict_index' is not defined

# 2. 입력 최종 테스트

In [13]:
emo_list = ["기쁨","분노","불안","슬픔"]  
env_list = ["가족관계", "건강","대인관계","연애, 결혼, 출산","재정","직장","학교폭력/따돌림"]

s1 = [input("문장을 입력해주세요 : ")]
cor_s1 = [okt.morphs(x) for x in s1]
seq_s1 = tokenizer.texts_to_sequences(cor_s1)
pad_s1 = preprocessing.sequence.pad_sequences(seq_s1,maxlen=MAX_SEQ_LEN)

emo_model_predict_proba = model.predict(pad_s1)[0]
emo_model_predict_class_index = emo_list[np.argmax(model.predict(pad_s1)[0])]
emo_model_predict_index = np.argmax(model.predict(pad_s1)[0])

env_model_predict_proba = env_model.predict(pad_s1)[0]
env_model_predict_class = env_list[np.argmax(env_model.predict(pad_s1)[0])]
env_model_predict_index = np.argmax(env_model.predict(pad_s1)[0])


request, response = emo_env_request(result,emo_model_predict_index,env_model_predict_index)
idx, answer = cosine_Answer(s1[0],request)
moa_answer = adaquate_answer(response,idx)

print("입력문 : ",s1[0])
print("유사문 : ", answer)
print("응답문 : ", moa_answer)
print("감정 : ",emo_model_predict_class_index)

문장을 입력해주세요 : 로또에 당첨이 되었어


NameError: name 'model' is not defined

# FLASK 연결

In [ ]:
from flask import Flask
from flask import request, redirect

app = Flask(__name__) 

@app.route('/review_model', methods=['GET','POST'])
def index():     
       
    if request.method == 'POST':
        #안드로이드에서 getParams()메소드 안에 작성한 
        # parsms.put("num1", "데이터1");
        
        # 의 key값인 num1, num2를 통해서 데이터 접근하여 변수에 저장
        num1 = str(request.form['message'])
        
        #num2 = int(request.form['num2'])
        
        
        
        #딥러닝 모델을 함수로 구현한 후 결과값을 return하여 result변수에 저장
        #딥러닝 모델은 아래 Cell에 만들 것
        result = model(num1)

    #결과값 리턴하면 안드로이드 스튜디오 StringRequest 안에 있는 onResponse()로 값 전달
    return result

if __name__ == '__main__':
    
    #ip주소 변경하세요
    app.run(host='172.30.1.59', port='5000') 

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.30.1.59:5000 (Press CTRL+C to quit)


1/1 [==============================] - 0s 13ms/step


172.30.1.53 - - [20/Jun/2022 11:09:18] "POST /review_model HTTP/1.1" 200 -


1/1 [==============================] - 0s 14ms/step


...

172.30.1.59 - - [20/Jun/2022 15:39:33] "POST /review_model HTTP/1.1" 200 -


1/1 [==============================] - 0s 14ms/step


172.30.1.59 - - [20/Jun/2022 15:39:45] "POST /review_model HTTP/1.1" 200 -


1/1 [==============================] - 0s 13ms/step


172.30.1.59 - - [20/Jun/2022 15:45:17] "POST /review_model HTTP/1.1" 200 -


# ★딥러닝 모델 코드를 아래 함수 안에 넣고 결과값을 return 문에 작성할 것

In [45]:
from tensorflow import keras
import numpy as np
import pandas as pd
import openpyxl
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense,Dropout, Conv1D,GlobalMaxPooling1D,concatenate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from konlpy.tag import Okt, Komoran


# def emo_env_request(df, emotion, enviroment):
#     condition1 = (df['감정_대분류']==emotion)
#     condition2 = (df['상황키워드']==enviroment)
#     Q_Answers = df[condition1 & condition2]
#     return Q_Answers['사람문장1'].to_list() , Q_Answers['시스템응답1'].to_list()

# # 감정문장 리스트 중 가장 유사한 문장을 찾아 인덱스를 반환하는 함수
# def cosine_Answer(txt,df_list):
#     cos_sim_list = [] 
#     ####### 텍스트 유사도 측정 ######
#     # 문장 벡터화 하기(사전 만들기)
#     for i in df_list:
#         tfidf_vectorizer = TfidfVectorizer()
#         sentences = (txt,i)
#         tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
#         ### 코사인 유사도 ###
#         cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
#         cos_sim_list.append(cos_similar[0][0])
#     # 가장 유사한 문장이 있는 인덱스와 가장 유사한 문장을 반환 
#     return np.argmax(cos_sim_list), df_list[np.argmax(cos_sim_list)]


# # 코사인 함수의 인덱스를 이용해 가장 적절한 응답을 찾는 함수 
# def adaquate_answer(df, idx):
#     return df[idx]




#머신러닝 모델함수
def model(num1):
    emo_model = keras.models.load_model("./best-emo-model.h5")
    env_model = keras.models.load_model("./best-env-model.h5")
    #모델구현
    s1 = [num1]
    cor_s1 = [okt.morphs(x) for x in s1]
    seq_s1 = tokenizer.texts_to_sequences(cor_s1)
    pad_s1 = preprocessing.sequence.pad_sequences(seq_s1,maxlen=100)

    emo_model_predict_proba = emo_model.predict(pad_s1)[0]
    emo_model_predict_class_index = emo_list[np.argmax(emo_model.predict(pad_s1)[0])]
    emo_model_predict_index = np.argmax(emo_model.predict(pad_s1)[0])
    


    env_model_predict_proba = env_model.predict(pad_s1)[0]
    env_model_predict_class = env_list[np.argmax(env_model.predict(pad_s1)[0])]
    env_model_predict_index = np.argmax(env_model.predict(pad_s1)[0])

    request, response = emo_env_request(result,emo_model_predict_index,env_model_predict_index)
    idx, answer = cosine_Answer(s1[0],request)
    moa_answer = adaquate_answer(response,idx)

    total_result = {"msg" : moa_answer, "msg_emo" : emo_model_predict_class_index}
     
    return total_result

In [53]:
model('건강이 좋지 않아')

1/1 [==============================] - 0s 13ms/step


{'msg': '건강에 대해 고민이 있으시네요. 좀 더 말씀해 보시겠어요?', 'msg_emo': '슬픔'}

In [32]:
import time
from datetime import datetime

now = datetime.now()

print("현재 : ", now)
print("현재 날짜 : ", now.date())

현재 :  2022-06-18 15:18:52.025769
현재 날짜 :  2022-06-18


In [42]:
!pip install pymysql

In [ ]:
pymysql.connect(host='project-db-stu.ddns.net', user='MOA', passoword="1234", db="MOA", charset='utf8')